<a href="https://colab.research.google.com/github/microsoft/FLAML/blob/main/notebook/autogen_agent_function_call.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Interative LLM Agent with Function Calls

FLAML offers an experimental feature of interactive LLM agents, which can be used to solve various tasks with human or automatic feedback, including tasks that require using tools via code. Please find documentation about this feature [here](https://microsoft.github.io/FLAML/docs/Use-Cases/Auto-Generation#agents-experimental).

In this notebook, we demonstrate how to use `AssistantAgent` and `UserProxyAgent` to make function calls with the new feature of OpenAI models (in model version 0613). A specified prompt and function configs need to be passed to `AssistantAgent` to initialize the agent. The corresponding functions need to be passed to `UserProxyAgent`, which will be responsible for executing any function calls made by `AssistantAgent`. Besides this requirement of matching descriptions with functions, we recommend checking the system prompt to make sure the instructions align with the function call descriptions.

## Requirements

FLAML requires `Python>=3.8`. To run this notebook example, please install flaml with the [mathchat] option since we will import functions from `MathUserProxyAgent`:
```bash
pip install flaml[mathchat]
```

In [1]:
# %pip install flaml[mathchat]~=2.0.0rc4

## Set your API Endpoint

The [`config_list_from_models`](https://microsoft.github.io/FLAML/docs/reference/autogen/oai/openai_utils#config_list_from_models) function tries to create a list of configurations using Azure OpenAI endpoints and OpenAI endpoints for the provided list of models. It assumes the api keys and api bases are stored in the corresponding environment variables or local txt files:

- OpenAI API key: os.environ["OPENAI_API_KEY"] or `openai_api_key_file="key_openai.txt"`.
- Azure OpenAI API key: os.environ["AZURE_OPENAI_API_KEY"] or `aoai_api_key_file="key_aoai.txt"`. Multiple keys can be stored, one per line.
- Azure OpenAI API base: os.environ["AZURE_OPENAI_API_BASE"] or `aoai_api_base_file="base_aoai.txt"`. Multiple bases can be stored, one per line.

It's OK to have only the OpenAI API key, or only the Azure OpenAI API key + base.
If you open this notebook in google colab, you can upload your files by click the file icon on the left panel and then choose "upload file" icon.

The following code excludes Azure OpenAI endpoints from the config list because they don't support functions yet. Remove the `exclude` argument after they do.

In [3]:
from flaml import oai

config_list = oai.config_list_from_models(model_list=["gpt-4", "gpt-3.5-turbo", "gpt-3.5-turbo-16k"], exclude="aoai")

The config list looks like the following:
```python
config_list = [
    {
        'model': 'gpt-4',
        'api_key': '<your OpenAI API key here>',
    },  # OpenAI API endpoint for gpt-4
    {
        'model': 'gpt-3.5-turbo',
        'api_key': '<your OpenAI API key here>',
    },  # OpenAI API endpoint for gpt-3.5-turbo
    {
        'model': 'gpt-3.5-turbo-16k',
        'api_key': '<your OpenAI API key here>',
    },  # OpenAI API endpoint for gpt-3.5-turbo-16k
]
```


## Making Function Calls

In this example, we demonstrate function call execution with `AssistantAgent` and `UserProxyAgent`. With the default system prompt of `AssistantAgent`, we allow the LLM assistant to perform tasks with code, and the `UserProxyAgent` would extract code blocks from the LLM response and execute them. With the new "function_call" feature, we define a new function using the pre-defined `execute_code` from `UserProxyAgent` and specify the description of the function in the OpenAI config. 

Then, the model has two paths to execute code:
1. Put the code blocks in the response. `UserProxyAgent` will extract and execute the code through `execute_code` method in the class.
2. As we put a function description to OpenAI config and register a function `exec_code` in `UserProxyAgent`, the model can also make function calls (will be put in `function_call` field of the API reply). `UserProxyAgent` will execute the function call through the registered `exec_code` method.

In [2]:
from flaml.autogen.agent import AssistantAgent, UserProxyAgent

oai_config = {
    "functions": [
        {
            "name": "execute_code",
            "description": "Receive a python code or shell script and return the execution result.",
            "parameters": {
                "type": "object",
                "properties": {
                    "code_type": {
                        "type": "string",
                        "description": "Code type, 'python' or 'sh'.",
                    },
                    "code": {
                        "type": "string",
                        "description": "Valid Python code to execute.",
                    }
                },
                "required": ["code_type", "code"],
            },
        }
    ],
    "config_list": config_list,
}
chatbot = AssistantAgent("chatbot", oai_config=oai_config)

# create a UserProxyAgent instance named "user"
user = UserProxyAgent(
    "user",
    human_input_mode="NEVER",
    code_execution_config={"work_dir": "coding"},
)

# define an `execute_code` function according to the function desription
def exec_code(code_type, code):
    # here we reuse the method in the user proxy agent
    # in general, this is not necessary
    return user.execute_code_blocks([(code_type, code)])

# register the `execute_code` function
user.register_function(function_map={"execute_code": exec_code})

# start the conversation
user.initiate_chat(
    chatbot,
    message="Draw a rocket and save to a file named 'rocket.svg'",
)


user (to chatbot):

Draw a rocket and save to a file named 'rocket.svg'

--------------------------------------------------------------------------------
chatbot (to user):

You can use the `svgwrite` library for Python to draw images into SVG format. Before we draw the rocket, you need to install the library. Use the code below to install it.

```sh
pip install svgwrite
```

After installing the library, here is the python code you can use to draw a rocket and save it to a file named 'rocket.svg':

```python
# filename: draw_rocket.py

import svgwrite

def draw_rocket():
    dwg = svgwrite.Drawing('rocket.svg', profile='tiny')

    # Draw rocket body
    dwg.add(dwg.rect((50, 20), (20, 40), fill='grey'))

    # Draw top of rocket
    dwg.add(dwg.polygon(points=[(50, 20), (60, 0), (70, 20)], fill='red'))

    # Draw bottom of rocket
    dwg.add(dwg.polygon(points=[(50, 60), (60, 80), (70, 60)], fill='red'))

    # Draw rocket window
    dwg.add(dwg.circle(center=(60, 40), r=5, fill='bl

user (to chatbot):

exitcode: 0 (execution succeeded)
Code output: 
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.1/67.1 kB 3.2 MB/s eta 0:00:00



--------------------------------------------------------------------------------
chatbot (to user):

***** Suggested function Call: execute_code *****
Arguments: 
{
  "code_type": "python",
  "code": "import svgwrite\n\ndef draw_rocket():\n    dwg = svgwrite.Drawing('rocket.svg', profile='tiny')\n\n    # Draw rocket body\n    dwg.add(dwg.rect((50, 20), (20, 40), fill='grey'))\n\n    # Draw top of rocket\n    dwg.add(dwg.polygon(points=[(50, 20), (60, 0), (70, 20)], fill='red'))\n\n    # Draw bottom of rocket\n    dwg.add(dwg.polygon(points=[(50, 60), (60, 80), (70, 60)], fill='red'))\n\n    # Draw rocket window\n    dwg.add(dwg.circle(center=(60, 40), r=5, fill='blue'))\n\n    dwg.save()\n\ndraw_rocket()"
}
*************************************************

--------------------------------------------------------------------------------


In [10]:
# uncomment the following to render the svg file
# from IPython.display import SVG, display

# display(SVG("coding/rocket.svg"))

## Another example with Wolfram Alpha API

We give another example of query Wolfram Alpha API to solve math problem. We use the predefined function `MathUserProxyAgent().execute_one_wolfram_query` as the function to be called.

In [5]:
import os
from flaml.autogen.agent import AssistantAgent, UserProxyAgent
from flaml.autogen.agent.math_user_proxy_agent import MathUserProxyAgent

# you need to provide a wolfram alpha appid to run this example
if not os.environ.get("WOLFRAM_ALPHA_APPID"):
    os.environ["WOLFRAM_ALPHA_APPID"] = open("wolfram.txt").read().strip()


sys_prompt = """You are an advanced AI with the capability to solve complex math problems.
Wolfram alpha is provided as an external service to help you solve math problems.

When the user gives a math problem, please use the most efficient way to solve the problem.
You are encouraged to use Wolfram alpha whenever it is possible during the solving process. For example, simplications, calculations, equation solving, etc.
However, if the operation requires little computation (very simple calculations, etc), you can also solve it directly.
Reply "TERMINATE" in the end when everything is done.
"""
oai_config = {
    "model": "gpt-4-0613",
    "functions": [
        {
            "name": "query_wolfram",
            "description": "Return the API query result from the Wolfram Alpha. the ruturn is a tuple of (result, is_success).",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "The Wolfram Alpha code to be executed.",
                    }
                },
                "required": ["query"],
            },
        }
    ],
    "config_list": config_list,
}
chatbot = AssistantAgent("chatbot", system_message=sys_prompt, oai_config=oai_config)

# the key in `function_map` should match the function name in "functions" above
# we register a class instance method directly
user = UserProxyAgent(
    "user",
    max_consecutive_auto_reply=2,
    human_input_mode="NEVER",
    function_map={"query_wolfram": MathUserProxyAgent().execute_one_wolfram_query},
)

# start the conversation
user.initiate_chat(
    chatbot,
    message="Problem: Find all $x$ that satisfy the inequality $(2x+10)(x+3)<(3x+9)(x+8)$. Express your answer in interval notation.",
)


user (to chatbot):

Problem: Find all $x$ that satisfy the inequality $(2x+10)(x+3)<(3x+9)(x+8)$. Express your answer in interval notation.

--------------------------------------------------------------------------------
chatbot (to user):

***** Suggested function Call: query_wolfram *****
Arguments: 
{
  "query": "solve (2x+10)(x+3) < (3x+9)(x+8) for x"
}
**************************************************

--------------------------------------------------------------------------------

>>>>>>>> NO HUMAN INPUT RECEIVED. USING AUTO REPLY FOR THE USER...
user (to chatbot):

***** Response from calling function "query_wolfram" *****
('Assumption: solve (2 x + 10) (x + 3)<(3 x + 9) (x + 8) for x \nAnswer: ans 0: x<-14\nans 1: x>-3\n', True)
**********************************************************

--------------------------------------------------------------------------------
chatbot (to user):

The solution to the inequality $(2x+10)(x+3)<(3x+9)(x+8)$ is $x \in (-\infty, -14) \cup (